This example is partialy structured : only some data is defined with keywords

In [1]:
from pprint import pprint
from datetime import date, time, timedelta, datetime
from json_ntv import Ntv, NtvTree
measure = {'location': [
                {'city': 'paris'}, 
                [4.1, 40.5]],
           'campaign': {
                'planning': {
                   'period': 'january 2023', 
                   'from': '2023-01-01', 
                   'to': '2023-01-20'},
                 'property': 'air pollutants',
                 'my personnal comment': {
                     'result': 'success',
                     'difficulty': 'first air pollutants campaign in Paris'}},
           'pollutants': [
                {'NO2': 'mcg/m3'}, 
                {'CO': 'mg/m3'}],
           'measurement': {
                'sample': '2023-01-10',
                'values': [
                   ['08:10:00', 21, 5],
                   ['10:20:00', 10, 10],
                   ['17:16:00', 30, 15]]}}
n_measure = Ntv.obj(measure)
t_measure = n_measure.tree.dic_nodes

## selecting data

### JSON selecting
- selector is row for the array and key for the objects

In [2]:
# access to the measurement time of the second value
print(measure['measurement']['values'][1][0])
# how to access to the 'result' of the 'campaign' (if i don't know the key)  ?

10:20:00


### NTV selecting
- selector is row or keys
- selector is a json-pointer
- selector is an absolute keys

In [3]:
# access to the measurement time of the second value
print(n_measure['measurement']['values'][1][0])
print(n_measure[3]['values'][1][0])
print(n_measure[3][1][1][0])
print(n_measure['/measurement/values/1/0'])
print(n_measure['/3/1/1/0'])
print(t_measure['values'].val[1][0])
# how to access to the 'period' of the 'campaign'  ?
print(t_measure['result'].val)
print(n_measure['campaign'][2]['result'].val)

"10:20:00"
"10:20:00"
"10:20:00"
"10:20:00"
"10:20:00"
"10:20:00"
success
success


## representation
- with values only
- with a tree

In [4]:
pprint(n_measure.to_obj(simpleval=True), width=140)


[['paris', [4.1, 40.5]],
 [['january 2023', '2023-01-01', '2023-01-20'], 'air pollutants', ['success', 'first air pollutants campaign in Paris']],
 ['mcg/m3', 'mg/m3'],
 ['2023-01-10', [['08:10:00', 21, 5], ['10:20:00', 10, 10], ['17:16:00', 30, 15]]]]


In [5]:
n_measure.to_mermaid('measure flowchart', disp=True)

## transform data
- change structure
- add semantic type

### Change structure

In [6]:
n_measure['location'][1] = n_measure['location'][1].to_single(name='coord', typ='point')
n_measure['measurement'].insert(0, n_measure['pollutants'])

### Add semantic type

In [7]:
t_measure['from'].set_type('date')
t_measure['to'].set_type('date')
t_measure['sample'].set_type('date')
for val in t_measure['values']:
    val[0].set_type('time')

In [8]:
pprint(n_measure.to_obj(), width=140)

{'campaign': {'my personnal comment': {'difficulty': 'first air pollutants campaign in Paris', 'result': 'success'},
              'planning': {'from:date': '2023-01-01', 'period': 'january 2023', 'to:date': '2023-01-20'},
              'property': 'air pollutants'},
 'location': {'city': 'paris', 'coord:point': [4.1, 40.5]},
 'measurement': {'pollutants': {'CO': 'mg/m3', 'NO2': 'mcg/m3'},
                 'sample:date': '2023-01-10',
                 'values': [[{':time': '08:10:00'}, 21, 5], [{':time': '10:20:00'}, 10, 10], [{':time': '17:16:00'}, 30, 15]]}}


In [9]:
n_measure.to_mermaid('add semantic types', disp=True)

## update data

In [10]:
# convert JSON value into object
o_measure = n_measure.to_obj_ntv()

In [11]:
# update objects
for ntv in o_measure.tree.leaf_nodes:
    if ntv.type_str == 'date':
        ntv.set_value(ntv.val + timedelta(days=1))
    if ntv.type_str == 'time':
        ntv.set_value((datetime.combine(date.today(), ntv.val) + timedelta(hours=1)).time())


In [12]:
# convert object into JSON value
n_measure = o_measure.to_json_ntv()

In [13]:
pprint(n_measure.to_obj(), width=140)

{'campaign': {'my personnal comment': {'difficulty': 'first air pollutants campaign in Paris', 'result': 'success'},
              'planning': {'from:date': '2023-01-02', 'period': 'january 2023', 'to:date': '2023-01-21'},
              'property': 'air pollutants'},
 'location': {'city': 'paris', 'coord:point': [4.1, 40.5]},
 'measurement': {'pollutants': {'CO': 'mg/m3', 'NO2': 'mcg/m3'},
                 'sample:date': '2023-01-11',
                 'values': [[{':time': '09:10:00'}, 21, 5], [{':time': '11:20:00'}, 10, 10], [{':time': '18:16:00'}, 30, 15]]}}


In [14]:
n_measure.to_mermaid('update data', disp=True)

### restore initial structure

In [15]:
n_measure.to_simple()
n_measure['location'][1] = n_measure.tree.dic_nodes['coord'].to_list()
n_measure['location'][1].set_name()
n_measure.insert(2, n_measure.tree.dic_nodes['pollutants'])

n_measure.to_mermaid('update data', disp=True)

In [16]:
pprint(n_measure.to_obj(), width=140)


{'campaign': {'my personnal comment': {'difficulty': 'first air pollutants campaign in Paris', 'result': 'success'},
              'planning': {'from': '2023-01-02', 'period': 'january 2023', 'to': '2023-01-21'},
              'property': 'air pollutants'},
 'location': [{'city': 'paris'}, [4.1, 40.5]],
 'measurement': {'sample': '2023-01-11', 'values': [['09:10:00', 21, 5], ['11:20:00', 10, 10], ['18:16:00', 30, 15]]},
 'pollutants': {'CO': 'mg/m3', 'NO2': 'mcg/m3'}}
